MultiLayer Perceptron for Titanic Data

In [2]:
# importing required libraries

import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


Loading the Data

In [3]:
train = pd.read_csv('/content/drive/Othercomputers/My Laptop/syr_ads_ist707/week10neural_network/lab10/titanic-train.csv')
train.shape

(891, 11)

In [4]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test = pd.read_csv('/content/drive/Othercomputers/My Laptop/syr_ads_ist707/week10neural_network/lab10/titanic-test.csv')
test.shape

(418, 11)

In [6]:
# converting cabin to binary attribute using dummy variable
train['Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# converting sex cateogorical variable to dummy variable
train['Sex'] = train["Sex"].replace({'male':0,'female':1})
test['Sex'] = test["Sex"].replace({'male':0,'female':1})

In [7]:
full_data = [train,test]
full_data

[     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch            Ticket  \
 0              1         0       3    0  22.0      1      0         A/5 21171   
 1              2         1       1    1  38.0      1      0          PC 17599   
 2              3         1       3    1  26.0      0      0  STON/O2. 3101282   
 3              4         1       1    1  35.0      1      0            113803   
 4              5         0       3    0  35.0      0      0            373450   
 ..           ...       ...     ...  ...   ...    ...    ...               ...   
 886          887         0       2    0  27.0      0      0            211536   
 887          888         1       1    1  19.0      0      0            112053   
 888          889         0       3    1   NaN      1      2        W./C. 6607   
 889          890         1       1    0  26.0      0      0            111369   
 890          891         0       3    0  32.0      0      0            370376   
 
         Fare 

In [8]:
# Remove all NULLS in the Age column
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

In [9]:
X_train = train[['Sex','Pclass','Age','SibSp','Parch','Cabin']]
Y_train = train['Survived']
X_test = test[['Sex','Pclass','Age','SibSp','Parch','Cabin']]

MPP

In [31]:
num_classes = 1

mlp_classifier = keras.Sequential(
    [
        keras.Input(shape=X_train.shape[1]),
        layers.Dense(9, activation='relu'),
        layers.Dense(9, activation='relu'),
        layers.Dense(5, activation='relu'),
        layers.Dense(num_classes, activation="sigmoid"), # similar to logistic regression in keras
    ]
)

mlp_classifier.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 9)                 63        
                                                                 
 dense_24 (Dense)            (None, 9)                 90        
                                                                 
 dense_25 (Dense)            (None, 5)                 50        
                                                                 
 dense_26 (Dense)            (None, 1)                 6         
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [33]:
batch_size = 128
epochs = 300

# compiling the ANN - basically setting up the optimizer
mlp_classifier.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fitting the ANN to the training data
mlp_classifier.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)

Epoch 1/300
7/7 [==============================] - 1s 31ms/step - loss: 0.4416 - accuracy: 0.8177 - val_loss: 0.3943 - val_accuracy: 0.8444
Epoch 2/300
7/7 [==============================] - 0s 9ms/step - loss: 0.4365 - accuracy: 0.8290 - val_loss: 0.3937 - val_accuracy: 0.8556
Epoch 3/300
7/7 [==============================] - 0s 9ms/step - loss: 0.4327 - accuracy: 0.8315 - val_loss: 0.3979 - val_accuracy: 0.8556
Epoch 4/300
7/7 [==============================] - 0s 6ms/step - loss: 0.4379 - accuracy: 0.8277 - val_loss: 0.3963 - val_accuracy: 0.8667
Epoch 5/300
7/7 [==============================] - 0s 7ms/step - loss: 0.4331 - accuracy: 0.8315 - val_loss: 0.3942 - val_accuracy: 0.8444
Epoch 6/300
7/7 [==============================] - 0s 18ms/step - loss: 0.4336 - accuracy: 0.8290 - val_loss: 0.3939 - val_accuracy: 0.8556
Epoch 7/300
7/7 [==============================] - 0s 8ms/step - loss: 0.4324 - accuracy: 0.8315 - val_loss: 0.3930 - val_accuracy: 0.8556
Epoch 8/300
7/7 [========

In [34]:
score = mlp_classifier.evaluate(X_train, Y_train, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.38406550884246826
Test accuracy: 0.8338944911956787


In [35]:
# Prediction
Y_pred = mlp_classifier.predict(X_test)

In [36]:
y_final = (Y_pred > 0.5).astype(int).reshape(X_test.shape[0])

output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_final})
output.to_csv('prediction.csv', index=False)
